In [22]:
import kaggle
import pandas as pd
from langchain_ollama import OllamaLLM
import sklearn

### Initialize LLM

Start Ollama Docker container (see Resources/docker.sh) and call the model specified in the model_id

In [16]:
model_id = "gemma3" # ollama
ollama_server_url = "http://localhost:11434"
llm = OllamaLLM(model = model_id, base_url = ollama_server_url)

### Read Data
Data are from the Kaggle Competition, Jigsaw - Agile Community Rules Classification.

Save the following files to the data folder:
- train.csv
- test.csv
- sample_submission.csv

In [ ]:
df = pd.read_csv("data/train.csv")

### Prompt LLM
Create a loop that first uses information from df to generate a prompt, supply it to the LLM, and then extract the output into the probability list.

In [ ]:
probability = []

for each_row in df.index:
    rule = df["rule"][each_row]
    body = df["body"][each_row]
    input = "Calculate the probability that a sample of text violates a rule where, 0 = rule is not violated to 1 = rule is violated. Here is the rule: " + rule + ". Here is the text: " + body + ". Structure your response as only a numeric probability ranging from 0.0 to 1.0."
    response = llm.invoke(input)
    probability.append(response)

Here's an example of the prompting system but with a single case:

In [ ]:
input = "Calculate the probability that a sample of text violates a rule where, 0 = rule is not violated to 1 = rule is violated. Here is the rule: " + df["rule"][0] + ". Here is the text: " + df["body"][0] + ". Structure your response as only a numeric probability ranging from 0.0 to 1.0."
response = llm.invoke(input)
response

'0.9\n'